# Chemistry Evaluation Analysis

This notebook processes and analyzes the results from a chemistry evaluation survey, which are stored in two separate CSV files: one for the **position model** and one for the **transition model**.

The primary goal is to calculate performance metrics based on expert feedback for a series of questions (Q1-Q6).

## Workflow

1.  **Position Model Evaluation**:
    *   Loads the `position_model.csv` file, skipping the metadata header.
    *   Cleans the data by removing empty rows and columns.
    *   Separates feedback Q6 from other questions scores.
    *   Calculates and displays summary statistics grouped by compound.

2.  **Transition Model Evaluation**:
    *   Loads the `transition_model.csv` file.
    *   Performs manual data corrections.
    *   Calculates and displays summary statistics for each question (Q1-Q6).
    *   Analyzes the accuracy of reactant prediction by parsing the "Correct reactants" and "Incorrect reactants" columns to determine an overall accuracy score.

In [ ]:
import pandas as pd
import os

In [ ]:
current_directory = os.getcwd()
working_folder = os.path.abspath(os.path.join(current_directory, '..'))
print(f"Working Directory: {working_folder}")

In [ ]:
# settings 
position_evaluation = working_folder + "/results/chemistry_evaluation/250909_position_model.csv"
transition_evaluation = working_folder + "/results/chemistry_evaluation/250909_transition_model.csv"

In [ ]:

# Load the CSV file, skipping the first 5 rows using the skiprows parameter
position = pd.read_csv(position_evaluation, skiprows=8)
position

In [ ]:
# Drop rows where all values are missing
position.dropna(how='all', inplace=True)
# Drop columns where all values are missing
position.dropna(how='all', axis=1, inplace=True)
position

In [ ]:
# remove all columns in which Position == "Q6"

position_q6_answers = position[position["Position"] == "Q6"]
position_q6_answers

In [ ]:
position_no_q6_answers = position[position["Position"] != "Q6"]
position_no_q6_answers['Q2'] = pd.to_numeric(position_no_q6_answers['Q2'])
position_no_q6_answers

In [ ]:
def analyze_question_scores(dataframe, question_column):
    """
    Groups a DataFrame by 'Compound', calculates summary statistics, 
    and returns the summary along with key metrics.

    Args:
        dataframe: The input DataFrame.
        question_column (str): The name of the question column to analyze.

    Returns:
        A tuple containing:
        - compound_summary (pd.DataFrame): The summary DataFrame per compound.
        - correct (float): The total sum of scores across all compounds.
        - all_rows (int): The total number of rows analyzed.
        - overall_average (float): The overall average score.
    """
    print(f"--- Analysis for: {question_column} ---")
    
    compound_summary = dataframe.groupby('Compound').agg(
        sum_of_scores=(question_column, 'sum'),
        num_rows=(question_column, 'size')
    )

    compound_summary['average_score'] = compound_summary['sum_of_scores'] / compound_summary['num_rows']
    
    correct = compound_summary["sum_of_scores"].sum()
    print(f"Total Sum of Scores: {correct}")
    
    all_rows = compound_summary["num_rows"].sum()
    print(f"Total Number of Rows: {all_rows}")
    
    overall_average = 0.0
    if all_rows > 0:
        overall_average = correct / all_rows
        print(f"Overall Average Score: {overall_average:.4f}")
    else:
        print("Overall Average Score: N/A (no rows to calculate)")
    
    print("-" * 20)
    
    return compound_summary, correct, all_rows, overall_average

compound_summary, correct, all_rows, overall_average = analyze_question_scores(position_no_q6_answers, "Q1")
compound_summary

In [ ]:
compound_summary, correct, all_rows, overall_average = analyze_question_scores(position_no_q6_answers, "Q2")
compound_summary

In [ ]:
compound_summary, correct, all_rows, overall_average = analyze_question_scores(position_no_q6_answers, "Q3")
compound_summary

In [ ]:
compound_summary, correct, all_rows, overall_average = analyze_question_scores(position_no_q6_answers, "Q4")
compound_summary

In [ ]:
compound_summary, correct, all_rows, overall_average = analyze_question_scores(position_no_q6_answers, "Q5")
compound_summary

# Transition

In [ ]:

import pandas as pd

# Load the CSV file, skipping the first 5 rows using the skiprows parameter
transition = pd.read_csv(transition_evaluation, skiprows=8)
transition

In [ ]:
# compound == LEi515, Position = 14 remove row because it is outside of the ontology.
index_to_drop = transition[(transition['Compound'] == 'LEI515') & (transition['Position'] == 14)].index
transition.drop(index_to_drop, inplace=True)
transition

In [ ]:
# Lei401, position 3, remove from correct reactants ",5" because the model sees as chemically invalid (even though its correct)
condition = (transition['Compound'] == 'LEI401') & (transition['Position'] == 3)
print(condition)
# Get the current value, replace the substring, and set it back
# This approach avoids potential SettingWithCopyWarning
current_value = transition.loc[condition, 'Correct reactants'].iloc[0]
print(current_value)
new_value = current_value.replace(',5', '')
print(new_value)
transition.loc[condition, 'Correct reactants'] = new_value
transition

In [ ]:
# group by compound get the sum of each question and number of rows per compound

def prediction_evaluation(transition_df, row_name):
    temp_transition = transition_df[transition_df[row_name] != "-"]
    temp_transition[row_name] = pd.to_numeric(temp_transition[row_name])
    compound_summary = temp_transition.groupby('Compound').agg(
        # Sum all numeric columns
        sum_of_scores=(row_name, 'sum'),  # Replace 'Score' with your actual score column name if different
        # Count the number of rows for each compound
        num_rows=(row_name, 'size')
    )

    compound_summary['average_score'] = compound_summary['sum_of_scores'] / compound_summary['num_rows']
    correct = compound_summary["sum_of_scores"].sum()
    print(correct)
    all = compound_summary["num_rows"].sum()
    print(all)
    print(correct/all)
    return compound_summary

prediction_evaluation(transition, "Q1")

In [ ]:
prediction_evaluation(transition, "Q2")

In [ ]:
prediction_evaluation(transition, "Q3")

In [ ]:
prediction_evaluation(transition, "Q4")

In [ ]:
prediction_evaluation(transition, "Q5")

In [ ]:
prediction_evaluation(transition, "Q6")

#### How often do we predict correct reactants?

In [ ]:
transition["Correct reactants"] = transition["Correct reactants"].replace("-", "")
transition

In [ ]:
transition["splitted_correct_reactants"] = transition["Correct reactants"].apply(lambda x: x.split(',') if x else [])
transition

In [ ]:
transition["Incorrect reactants"] = transition["Incorrect reactants"].fillna('')
transition["splitted_incorrect_reactants"] = transition["Incorrect reactants"].apply(lambda x: x.split(',') if x else [])
transition

In [ ]:
transition["splitted_correct_reactants_len"] = transition["splitted_correct_reactants"].apply(lambda x: len(x))
transition["splitted_incorrect_reactants_len"] = transition["splitted_incorrect_reactants"].apply(lambda x: len(x))
transition

In [ ]:
transition["splitted_correct_reactants_len"].describe()

In [ ]:
transition["splitted_incorrect_reactants_len"].describe()

In [ ]:
compound_analysis = transition.groupby('Compound').agg(
    total_correct_reactants=('splitted_correct_reactants_len', 'sum'),
    total_incorrect_reactants=('splitted_incorrect_reactants_len', 'sum')
)
compound_analysis["total_reactants"] = compound_analysis["total_correct_reactants"] + compound_analysis["total_incorrect_reactants"]
compound_analysis

In [ ]:
correct = compound_analysis["total_correct_reactants"].sum()
print(correct)
total = compound_analysis["total_reactants"].sum()
print(total)
accuracy = correct / total
accuracy
